In [1]:
from future.backports.datetime import datetime
from misc.misc import run_in_thread
%load_ext autoreload
%autoreload 2
import matplotlib
matplotlib.use('TKAgg')
from pianoq.lab.mplc.mask_aligner import MPLCAligner
from pianoq.lab.pco_camera import PCOCamera
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.mplc_device import MPLCDevice
import os 
import datetime
import time 
dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import glob 
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots
import numpy as np 
modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])

mplc = MPLCDevice()
mplc.restore_location()
print('Got MPLC')
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_10_14_00_34_old.masks"
masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_25_13_19_15_extra_1cm.masks"
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_17_first_diffuser\2024_09_17_12_19_58_diffuser_d1d.masks"

msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
phases = np.random.uniform(0, 2*np.pi, size=50)
masks = add_phase_input_spots(masks, phases)
mplc.load_masks(masks, linear_tilts=True)
mplc.restore_location()

In [2]:
backlash = 0
wait_after_move = 0.3
from pianoq.lab.mplc.consts import thorlabs_x_serial, thorlabs_y_serial
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.zaber_motor import ZaberMotors
# zaber_ms = ZaberMotors(backlash=backlash, wait_after_move=wait_after_move)
# mxs = zaber_ms.motors[1]
# mys = zaber_ms.motors[0]
# print('Got Zaber motors')

mxi = ThorlabsKcubeDC(thorlabs_x_serial, backlash=backlash, wait_after_move=wait_after_move)
myi = ThorlabsKcubeStepper(thorlabs_y_serial, backlash=backlash, wait_after_move=wait_after_move)
print('Got Thorlabs motors')

Got Thorlabs motors


In [3]:
# Timetagger
from pianoq.lab.time_tagger import QPTimeTagger
from pianoq.lab.mplc.consts import TIMETAGGER_DELAYS, TIMETAGGER_COIN_WINDOW
tt = QPTimeTagger(integration_time=1, remote=True,
                  single_channel_delays=TIMETAGGER_DELAYS, coin_window=TIMETAGGER_COIN_WINDOW)

In [25]:
mxi.move_absolute(9.31)
myi.move_absolute(3.68)
print(mxi.get_position())
print(myi.get_position())

9.31
3.68


In [34]:
from pianoq.lab.photon_scan import PhotonScanner
from pianoq.misc.misc import run_in_thread

integration_time = 1
start_x = 9.0
end_x = 9.6
start_y = 3.75
end_y = 5.55

mxi.wait_after_move = 0
myi.wait_after_move = 0
# start_x = 8.5
# end_x = 9.5
# start_y = 2
# end_y = 4

resolution = 1 
x_pixels = 24 // resolution
y_pixels = 72 // resolution
pixel_size_x = 0.025 * resolution
pixel_size_y = 0.025 * resolution

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
singles_idler_path = fr'{dir_path}\{timestamp}_idler_5_spots_tilt_35_deadzone_450_fine.scan'

scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=2e-9, saveto_path=singles_idler_path)   
_ = scanner.scan(x_motor=mxi, y_motor=myi, ph=tt)

Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 521. Singles2: 502. Coincidence: 0.00.
dur: 1. pix: 0, 1. Singles1: 512. Singles2: 503. Coincidence: 0.00.
dur: 3. pix: 0, 2. Singles1: 504. Singles2: 491. Coincidence: 0.00.
dur: 4. pix: 0, 3. Singles1: 560. Singles2: 519. Coincidence: 0.00.
dur: 6. pix: 0, 4. Singles1: 599. Singles2: 502. Coincidence: 0.00.
dur: 8. pix: 0, 5. Singles1: 537. Singles2: 508. Coincidence: 0.00.
dur: 9. pix: 0, 6. Singles1: 588. Singles2: 507. Coincidence: 0.00.
dur: 11. pix: 0, 7. Singles1: 694. Singles2: 467. Coincidence: 0.00.
dur: 12. pix: 0, 8. Singles1: 698. Singles2: 508. Coincidence: 0.00.
dur: 14. pix: 0, 9. Singles1: 784. Singles2: 503. Coincidence: 0.00.
dur: 15. pix: 0, 10. Singles1: 753. Singles2: 480. Coincidence: 0.00.
dur: 17. pix: 0, 11. Singles1: 690. Singles2: 524. Coincidence: 0.00.
dur: 18. pix: 0, 12. Singles1: 703. Singles2: 523. Coincidence: 0.00.
dur: 20. pix: 0, 13. Singles1: 698. Singles2: 515. Coincidence